# 1. 环境配置

## 1.1 python 环境准备

In [ ]:
! pip install gradio==6.1.0 openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3 beautifulsoup4==4.14.3 langchain_chroma==1.1.0

## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

## 1.3 RAG 应用简介

RAG（Retrieval Augmented Generation）是一种结合了检索（Retrieval）和生成（Generation）的技术，旨在通过利用外部知识库来增强大型语言模型（LLMs）的性能。它通过检索与用户输入相关的信息片段，并结合这些信息来生成更准确、更丰富的回答。

虽然大语言模型（如 ChatGPT、Qwen 等）在生成能力上非常强大，但它们存在两大天然缺陷：
- 记忆有限：模型只能记住训练时看到的数据，无法“实时学习”新知识。
- 幻觉问题：模型可能会编造事实或给出虚假的信息（称为 hallucination），尤其在面对冷门或专业问题时。

# 2. RAG 基础流程演示

## 2.1 文档载入
在文档内容被制作成向量数据库之前，我们需要先收集相关的内容：
- 从PDF、数据库、URLs等不同来源读取内容（目前一般只读取文本的相关内容）；
- 然后统一转成Documents格式，作为后续“切分—嵌入—检索”的输入。

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://zh.d2l.ai/chapter_introduction/index.html")
docs = loader.load()
print(docs[0].page_content[:500]) # 将网页里面前500个字符打印出来

## 2.2 文档切分
文档切分是指将一个长文本的 Document 拆成若干个更小的段落（Chunks），每个段落大小适合被大模型理解和向量化处理。之所以要切分是因为：
- 大模型通常有 Token 长度限制（如 2048 / 4096 tokens），原始文档太长无法直接处理，必须拆分。
- 长文本容易造成语义漂移，而小段文本可以更聚焦地表达一个意思，有利于后续检索和匹配。

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 1500,
  chunk_overlap = 150)
splits = text_splitter.split_documents(docs)
print(len(splits))

## 2.3 向量数据库存储
在将每一个文档切割成合适的 chunk 后，我们还需要进行文本嵌入及向量数据库存储：
- 文本嵌入：使用 Embedding 模型将文本转换成高维向量（如 1536 维）。
- 向量存储：将向量和元数据（metadata）一起存入向量数据库中。

In [ ]:
from langchain_community.embeddings import DashScopeEmbeddings
import os

# 设置 embedding 模型（阿里云）
embeddings = DashScopeEmbeddings(
    dashscope_api_key=os.getenv('DASHSCOPE_API_KEY'), 
    model="text-embedding-v1")

# 设置文本内容
text_1 = "今天天气不错"

# 进行文本向量化
query_result = embeddings.embed_query(text_1)
print(query_result)

In [ ]:
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_chroma import Chroma
import os

embeddings = DashScopeEmbeddings(
    dashscope_api_key=os.getenv('DASHSCOPE_API_KEY'), 
    model="text-embedding-v1")

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory='./chroma')

print(vectordb._collection.count())

## 2.4 提问+向量数据库检索

然后我们就到了第二个阶段—提问 + 调用数据库。第一步，我们是需要根据用户提出的问题，在向量数据库里找到最相关的片段然后传给提示词。其内部原理如下：
- 首先要通过 Chorma 连接上当前的向量数据库。
- 接着将用户提问的内容用相同的 embedding 模型转为向量。
- 然后用余弦相似度计算它们的相似程度，相似度越高，代表语义越相关。
- 最后根据 Top-K 的值选择最相关的前 K 条内容进行返回。

那在 LangChain 的检索中通常有两种方式，一种是基本的相似度搜索，另外一种是进阶的 mmr 搜索：

### 2.4.1 相似度搜索

计算余弦相似度的方式进行检索：

In [ ]:
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_chroma import Chroma
import os

# 使用当前创建的向量数据库
embeddings = DashScopeEmbeddings(
 dashscope_api_key=os.getenv('DASHSCOPE_API_KEY'), 
 model="text-embedding-v1")
vectordb = Chroma(persist_directory="./chroma", embedding_function=embeddings)

# 设置问题
question = "日常生活中的机器学习"

# 利用相似度搜索检索与问题最相关的3个切片
retriever = vectordb.as_retriever(
  search_type="similarity", 
  search_kwargs={"k": 3})
docs = retriever.invoke(question)

# 打印第一个最相关的切块内容
print(docs[0].page_content)

### 2.4.2 最大边际相关性搜索

在向量数据库中进行信息检索时，除了相似度搜索以外，常见还有最大边际相关性（Maximum Marginal Relevance, MMR）的方法：

最大边际相关性是一种在信息检索中用于平衡 相关性 和 多样性 的技术，特别适用于需要避免重复内容并提高信息覆盖面的场景。它在传统的基于相似度的检索方法上进行了扩展，旨在通过同时考虑文档与查询的相关性以及文档之间的多样性来优化检索结果。
- 相关性（Relevance）：文档与查询之间的相似度，反映了文档对查询的相关性。
- 多样性（Diversity）：文档之间的相似度，旨在避免返回重复或冗余的内容。

In [ ]:
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_chroma import Chroma
import os

# 使用当前创建的向量数据库
embeddings = DashScopeEmbeddings(
 dashscope_api_key=os.getenv('DASHSCOPE_API_KEY'), 
 model="text-embedding-v1")
vectordb = Chroma(persist_directory="./chroma", embedding_function=embeddings)

# 设置问题
question = "日常生活中的机器学习"

# 利用相似度搜索检索与问题最相关的3个切片
retriever = vectordb.as_retriever(
  search_type="mmr", 
  search_kwargs={"k": 5, "fetch_k": 10, "lambda_mult": 0.25})
docs = retriever.invoke(question)

# 打印第一个最相关的切块内容
print(docs[0].page_content)

## 2.5 提示词模版+大模型回复
然后我们将检索到的相关内容（上下文）与用户问题，组织成一个 Prompt（提示词）。最后交给大模型进行理解和生成回答。

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 构建提示词模版
template = """请使用以下上下文信息回答最后的问题。
如果您不知道答案，就直接说您不知道，不要试图编造答案。
回答最多使用三句话。请尽可能简洁地回答。最后一定要说“谢谢提问！”。
上下文：{context}
问题：{question}
有帮助的回答："""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm = ChatOpenAI(model="ernie-4.0-turbo-128k",
  openai_api_key=os.environ.get("OPENAI_API_KEY"),
  base_url="https://aistudio.baidu.com/llm/lmapi/v3")


def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

retriever = vectordb.as_retriever(search_type="mmr", 
 search_kwargs={"k": 1, "fetch_k": 10, "lambda_mult": 0.25})

qa_chain = (
  {"context": retriever | format_docs,
    "question": RunnablePassthrough()}
  | QA_CHAIN_PROMPT
  | llm
  | StrOutputParser())

print(qa_chain.invoke("日常生活里，哪里用到了机器学习呢？"))

## 2.6 前端页面制作
在设计完RAG系统后，我们也来看看如何来设计前端页面：
- 从前面的流程可知，RAG系统分为两步，一步是生成向量数据库，下一步才是对话，所以在RAG的前端我们需要有上传内容的组件，并且需要通过点击按钮的方式生成我们的向量数据库。
- 然后我们就还是要设计一个聊天的页面，里面和之前一样要记录下来完整的聊天记录，并且我们也要有内容输入的页面。

In [ ]:
import gradio as gr
with gr.Blocks() as demo:
  gr.Markdown('# 基础 RAG 对话平台')
  with gr.Row():
    # 创建左侧列
    with gr.Column():
      # 创建一个文本框接收网页地址
      url = gr.Textbox(label='请输入网页地址')
      url_loader_button = gr.Button('点击生成向量数据库')
      # 创建一个文件上传组件接收文件
      document = gr.File(label='请上传文件')
      document_loader_button = gr.Button('点击生成向量数据库')
      # 创建一个 Markdown 块接收数据库生成的情况
      information = gr.Markdown()

    # 创建右侧列
    with gr.Column():
      # 创建一个 Chatbot 组件记录聊天内容
      chat_history = gr.Chatbot(label='聊天记录')
      # 创建一个文本区域记录要问的问题
      input_message = gr.TextArea(label='内容输入')
      state = gr.State([])
demo.launch()

这里面有三个事件触发器，需要我们一一进行函数的补充：
- load_web_content：加载链接类的文档并载入向量数据库
- load_document：加载本地上传的文档并载入向量数据库
- chat：基于创建好的向量数据库进行对话

### 2.6.1 load_web_content()

核心功能：将传入的网页链接里的内容切分后转为向量数据库的一部分：
- 输入：url 链接
- 输出：无输出或文字信息

In [ ]:
from langchain_community.document_loaders import (
  WebBaseLoader,
  YoutubeLoader)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_chroma import Chroma
import os

# 初始化向量数据库
persist_directory='./chroma'
embeddings = DashScopeEmbeddings(
 dashscope_api_key=os.getenv('DASHSCOPE_API_KEY'), 
 model="text-embedding-v1")
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
# 初始化文本切分方法
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
 chunk_overlap=150)

def load_web_content(url: str):
  # 判断内容类型
  if 'youtube.com' in url or 'youtu.be' in url:
    loader = YoutubeLoader(url)
  else:
    loader = WebBaseLoader(url)
  docs = loader.load()
  # 文本切分
  splits = text_splitter.split_documents(docs)
  vectordb = Chroma.from_documents(
    documents=splits,
    persist_directory=persist_directory,
    embedding=embeddings)
  return f"已成功在 {persist_directory} 文件夹生成向量数据库"


### 2.6.2 load_document()

核心功能：将传入的文件里的内容切分后转为向量数据库的一部分
- 输入：不同格式的文件
- 输出：无输出或文字信息

In [ ]:
from langchain_community.document_loaders import (
  TextLoader,
  PyMuPDFLoader,
  Docx2txtLoader,
  UnstructuredMarkdownLoader,
  UnstructuredExcelLoader)

def load_document(file_path: str):
 # 获取文件后缀名
 file_type = file_path.split('.')[-1].lower()
 # 根据文件类型选择合适的 Loader
 if file_type == 'txt':
  loader = TextLoader(file_path)
 elif file_type == 'pdf':
  loader = PyMuPDFLoader(file_path)
 elif file_type == 'docx':
  loader = Docx2txtLoader(file_path)
 elif file_type == 'md':
  loader = UnstructuredMarkdownLoader(file_path)
 elif file_type == 'xlsx':
  loader = UnstructuredExcelLoader(file_path)
 else:
  raise ValueError(f"不支持的文件类型: {file_type}")
 # 加载文档
 docs = loader.load()
 # 文本切分
 splits = text_splitter.split_documents(docs)
 vectordb = Chroma.from_documents(
  documents=splits,
  persist_directory=persist_directory,
  embedding=embeddings)
 return f"已成功在 {persist_directory} 文件夹生成向量数据库"

### 2.6.3 load_document()

核心功能：将问题和历史记录传入大模型，获得回复后将更新后的聊天记录传入聊天框中，并清空输入框的内容
- 输入：
    - 用户提问
    - 历史记录（前端 gr.State 保存）
- 输出：
    - 传入 chatbot 的历史记录
    - 更新 gr.State 的历史记录
    - 作用于输入框的空字符串

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

def chat(question, chat_history):
 # 初始化聊天模型
 llm = ChatOpenAI(model="ernie-4.0-turbo-128k",
  openai_api_key=os.environ.get("OPENAI_API_KEY"),
  base_url="https://aistudio.baidu.com/llm/lmapi/v3")
 # 构建提示词模版
 template = """请使用以下上下文信息回答最后的问题。
  如果您不知道答案，就直接说您不知道，不要试图编造答案。
  回答最多使用三句话。请尽可能简洁地回答。最后一定要说“谢谢提问！”。
  上下文：{context}
  问题：{question}
  有帮助的回答："""
 QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
 retriever = vectordb.as_retriever(search_type="mmr", 
  search_kwargs={"k": 1, "fetch_k": 10, "lambda_mult": 0.25})
 qa_chain = (
  {"context": retriever | format_docs,
    "question": RunnablePassthrough()}
  | QA_CHAIN_PROMPT
  | llm
  | StrOutputParser())
 result = qa_chain.invoke({question}) 
 chat_history.append({"role": "user", "content": question})
 chat_history.append({"role": "assistant", "content": result})
 return chat_history, chat_history, ""

### 2.6.4 页面整合

最后将三个函数进行整合：

In [ ]:
import gradio as gr
with gr.Blocks() as demo:
  gr.Markdown('# 基础 RAG 对话平台')
  with gr.Row():
    # 创建左侧列
    with gr.Column():
      # 创建一个文本框接收网页地址
      url = gr.Textbox(label='请输入网页地址')
      url_loader_button = gr.Button('点击生成向量数据库')
      # 创建一个文件上传组件接收文件
      document = gr.File(label='请上传文件')
      document_loader_button = gr.Button('点击生成向量数据库')
      # 创建一个 Markdown 块接收数据库生成的情况
      information = gr.Markdown()
      url_loader_button.click(fn = load_web_content, inputs= url , outputs= information)
      document_loader_button.click(fn = load_document, inputs = document, outputs = information)

    # 创建右侧列
    with gr.Column():
      # 创建一个 Chatbot 组件记录聊天内容
      chat_history = gr.Chatbot(label='聊天记录')
      # 创建一个文本区域记录要问的问题
      input_message = gr.TextArea(label='内容输入')
      state = gr.State([])
      input_message.submit(fn = chat, inputs=[input_message, state],outputs=[chat_history, state, input_message])
demo.launch()